In [17]:
from pymongo import MongoClient
import os
from pprint import pprint
import json

In [2]:
mongo_username = "root"
mongo_password = os.environ.get("PW")
host = "localhost"
port = "27020"  # prod

mongoclient_prod = MongoClient(
    "mongodb://%s:%s@%s:%s" % (mongo_username, mongo_password, host, port)
)

In [ ]:
mongo_username = "root"
mongo_password = os.environ.get("PW")
host = "localhost"
port = "27019"  # dev

mongoclient_dev = MongoClient(
    "mongodb://%s:%s@%s:%s" % (mongo_username, mongo_password, host, port)
)

In [2]:
host = "localhost"
port = "27017"  # local

mongoclient_local = MongoClient("mongodb://%s:%s" % (host, port))

## For Tornado


In [ ]:
def add_field(mongodbclient: MongoClient) -> None:
    for document in mongodbclient["hazarddb"]["TornadoDataset"].find():
        document["hazardDatasets"] = [
            {
                "datasetId": document["datasetId"],
                "demandType": "wind",
                "demandUnits": "mph",
                "threshold": None,
            }
        ]
        # replace document
        mongodbclient["hazarddb"]["TornadoDataset"].replace_one(
            {"_id": document["_id"]}, document
        )
        # pprint(document)

In [5]:
## for local
add_field(mongoclient_local)

## for prod
# add_field(mongoclient_prod)

## for dev
# add_field(mongoclient_dev)

In [ ]:
def del_datasetId(mongodbclient: MongoClient) -> None:
    revert_dict: dict = {}
    for document in mongodbclient["hazarddb"]["TornadoDataset"].find():
        # remove datasetId key from main document
        revert_dict[document["_id"]] = document["datasetId"]
        del document["datasetId"]
        # replace document
        mongodbclient["hazarddb"]["TornadoDataset"].replace_one(
            {"_id": document["_id"]}, document
        )
    with open("revert_deleted_datasetIds.json", "w") as f:
        f.write(json.dumps(revert_dict, indent=4))

In [ ]:
def revert_del_datasetIds(mongodbclient: MongoClient) -> None:
    with open("revert_deleted_datasetIds.json", "r") as f:
        revert_dict = json.load(f)
    for _id, datasetId in revert_dict.items():
        mongodbclient["hazarddb"]["TornadoDataset"].update_one(
            {"_id": _id}, {"$set": {"datasetId": datasetId}}
        )

In [ ]:
### run after we have updated all other components and we are ready to remove this key.

## for local
# del_datasetId(mongoclient_local)

## for prod
# del_datasetId(mongoclient_prod)

## for dev
# del_datasetId(mongoclient_dev)

In [ ]:
### NOOO!!! revertttttt

## for local
# revert_del_datasetIds(mongoclient_local)

## for prod
# revert_del_datasetIds(mongoclient_prod)

## for dev
# revert_del_datasetIds(mongoclient_dev)

## For Earthquake


In [15]:
def modify_field(mongodbclient: MongoClient) -> None:
    for document in mongodbclient["hazarddb"]["EarthquakeModel"].find():
        # modify field
        document["hazardDataset"] = document["rasterDataset"]
        document.pop("rasterDataset")
        # replace document
        mongodbclient["hazarddb"]["EarthquakeModel"].replace_one(
            {"_id": document["_id"]}, document
        )

In [16]:
## for local
modify_field(mongoclient_local)

## for prod
# modify_field(mongoclient_prod)

## for dev
# modify_field(mongoclient_dev)